In [ ]:
# категориальные
group_cat = ['is_apartment', 'has_elevator', 'rooms', 'building_type_int'] 
# высококардинальные категориальные, или количественные, ассимметричные 
group_num_floors = ['floor', 'floors_total', 'ceiling_height', 'flats_count'] 
# количественные, ассимметричные
group_num_area = ['kitchen_area','living_area','total_area'] 
# количественные 
group_num_geo = ['latitude', 'longitude'] 
# временной ряд
group_num_date = ['building_id','build_year'] 
# новые 
group_new = ['distance']

In [ ]:
def delete_all_combinations_metrics(data, exclude_cols, sort='rmse'):
    def all_combinations(iterable):
        unique_subsets = set()
        for r in range(len(iterable)+1):
            for subset in combinations(iterable, r):
                unique_subsets.add(subset)
        return unique_subsets
        
    x_train, y_train, x_val, y_val = data
    comb_list = list(all_combinations(exclude_cols))

    df_res = pd.DataFrame(columns=['mae', 'rmse', 'r2', 'rest_columns'])
    df_res.index.name='removed_columns'

    for exclude_subset in tqdm(comb_list, total=len(comb_list), colour='blue'):
        include_cols = [col for col in x_train.columns if col not in exclude_subset]
        subset = x_train[include_cols], y_train, x_val[include_cols], y_val
        metrics = get_metrics(subset)
        metrics['rest_columns'] =  tuple(col for col in exclude_cols if col not in exclude_subset)
        df_res.loc[str(exclude_subset)] = metrics
    
    return df_res.sort_values(by=sort)

data = X_train, y_train, X_val, y_val
delete_all_combinations_metrics(data, exclude_cols=group_cat + group_new, sort='rmse')